## Overview

__This notebook is about the first phase where initial Cleaning on the data is done. The main task was to transform the data into transaction string with respect to the user session ID and aggregate them into a single transaction (in the format that the apriori algorithm will require the data to be)__

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
from pyspark.sql.types import *

In [0]:
# Ref: https://spark.apache.org/docs/latest/sql-ref-datatypes.html
file_schema = StructType([
    StructField('event_time', StringType(), False),
    StructField('event_type', StringType(), True),
    StructField('product_id', IntegerType(), True),
    StructField('category_id', LongType(), True),
    StructField('category_code', StringType(), True),
    StructField('brand', StringType(), True),
    StructField('price', FloatType(), True),
    StructField('user_id', IntegerType(), True),
    StructField('user_session', StringType(), True)
])

In [0]:
# File location and type
file_location = "/FileStore/tables/chunk1-1.csv"
file_type = "csv"

# CSV options
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, schema= file_schema)
df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 10:22:...|      view|  35000008|2070005009172398851|                null| koopman|  64.33|512377508|1fa73df9-c28f-427...|
|2019-10-01 10:22:...|      cart|   1005113|2053013555631882655|electronics.smart...|   apple|1027.05|514816113|24e7fe12-1cee-455...|
|2019-10-01 10:22:...|      view|  31501161|2053013558031024687|                null|luminarc| 102.71|533329645|83769917-68de-439...|
|2019-10-01 10:22:...|      view|   6000264|2053013560807654091|auto.accessories....|starline| 310.43|514453000|d3972d16-00c6-48d...|
|2019-10-01 10:22:...|      cart|  12700931|205301355355989635

In [0]:
df.show(truncate=False)

+-----------------------+----------+----------+-------------------+---------------------------------+--------+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                    |brand   |price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+---------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 10:22:43 UTC|view      |35000008  |2070005009172398851|null                             |koopman |64.33  |512377508|1fa73df9-c28f-4270-883c-852ae7960adc|
|2019-10-01 10:22:43 UTC|cart      |1005113   |2053013555631882655|electronics.smartphone           |apple   |1027.05|514816113|24e7fe12-1cee-455d-aa59-73df65482ffe|
|2019-10-01 10:22:43 UTC|view      |31501161  |2053013558031024687|null                             |luminarc|102.71 |533329645|83769917-68de-439d-aade-43d2268a68e4|
|201

In [0]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c)|col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|event_time|event_type|product_id|category_id|category_code|brand|price|user_id|user_session|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+
|         0|         0|         0|          0|       159692|75709|    0|      0|           0|
+----------+----------+----------+-----------+-------------+-----+-----+-------+------------+



In [0]:
(df.select('event_type')
#.where(col('category_code').isNotNUll())
.groupBy('event_type')
.count()
.show(5, False))

+----------+------+
|event_type|count |
+----------+------+
|purchase  |7090  |
|view      |486680|
|cart      |6230  |
+----------+------+



In [0]:
import pyspark.sql.functions as F

df.groupby('event_type').agg(
    (F.count('event_type')).alias('count'),
    (F.count('event_type')*100 / df.count()).alias('percentage')
).show()

+----------+------+----------+
|event_type| count|percentage|
+----------+------+----------+
|  purchase|  7090|     1.418|
|      view|486680|    97.336|
|      cart|  6230|     1.246|
+----------+------+----------+



## __Main Tranformation Code Block__

In [0]:
#DataFlow

# Compress the data and store in an optimized format.


#Missing Values
nonNAdf = df.where(df.category_code.isNotNull())


#Separate_productTypes
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col

def string_split(string1):
    main_string = string1.split('.')[-1]
    main_string = main_string.strip()
    return main_string

udfstringfunc = F.udf(string_split, StringType())
nonNAdf = nonNAdf.withColumn('productType', udfstringfunc('category_code'))



# Events Split
nonNAdf_purchase = nonNAdf.select('*').where(col('event_type')=='purchase')
nonNAdf_view = nonNAdf.select('*').where(col('event_type')=='view')
nonNAdf_cart = nonNAdf.select('*').where(col('event_type')=='cart')

#groupby users Purchase
grouped_transactions = nonNAdf_purchase.select('*').groupBy('user_session').agg(F.concat_ws(",",F.collect_list(nonNAdf_purchase.productType))).select(col('user_session'), col("concat_ws(,, collect_list(productType))").alias("transactions"))


#join the table
grouped_transactions = grouped_transactions.join(nonNAdf_purchase, grouped_transactions.user_session==nonNAdf_purchase.user_session, 'left')

#select the required columns
(grouped_transactions.select(col('event_time'), col('event_type'), col('product_id'), col('productType'), col('transactions'), col('user_id'), col('brand')).show(truncate=False))



# Time conversion 



+-----------------------+----------+----------+-----------+-------------------------------------------+---------+-------+
|event_time             |event_type|product_id|productType|transactions                               |user_id  |brand  |
+-----------------------+----------+----------+-----------+-------------------------------------------+---------+-------+
|2019-10-01 13:37:06 UTC|purchase  |5100564   |clocks     |clocks                                     |520206104|samsung|
|2019-10-01 14:13:11 UTC|purchase  |1005130   |smartphone |smartphone                                 |516675926|apple  |
|2019-10-01 14:46:57 UTC|purchase  |2501075   |oven       |oven,oven,oven                             |514242794|artel  |
|2019-10-01 14:45:55 UTC|purchase  |2501244   |oven       |oven,oven,oven                             |514242794|artel  |
|2019-10-01 14:44:16 UTC|purchase  |2501143   |oven       |oven,oven,oven                             |514242794|artel  |
|2019-10-01 13:31:01 UTC

### Exaplantion on the transformation that is done with code.

In [0]:
def string_split(string1):
    main_string = string1.split('.')[-1]
    return main_string

In [0]:
# Before we split the string category_code, we have to either drop missing values from category_code.
#here I am not dropping the values thinking that the data other than just category_code can be helpful in any way.
# so will create a new df 

nonNAdf = df.where(df.category_code.isNotNull())
nonNAdf.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 10:22:...|      cart|   1005113|2053013555631882655|electronics.smart...|   apple|1027.05|514816113|24e7fe12-1cee-455...|
|2019-10-01 10:22:...|      view|   6000264|2053013560807654091|auto.accessories....|starline| 310.43|514453000|d3972d16-00c6-48d...|
|2019-10-01 10:22:...|      view|  41900000|2090971686529663114|appliances.enviro...|    null|   23.6|551906402|a8f2500d-1af3-495...|
|2019-10-01 10:22:...|      view|  19200082|2053013556202308035|construction.tool...|  p.i.t.| 171.43|546270188|2f7b4975-5212-46a...|
|2019-10-01 10:22:...|      view|   6200315|205301355229321647

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def string_split(string1):
    main_string = string1.split('.')[-1]
    return main_string

#Register the UserDefined Function
udfstringfunc = F.udf(string_split, StringType())
df_with_productType=nonNAdf.withColumn('productType', udfstringfunc('category_code'))

In [0]:
df_with_productType.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|productType|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+
|2019-10-01 10:22:...|      cart|   1005113|2053013555631882655|electronics.smart...|   apple|1027.05|514816113|24e7fe12-1cee-455...| smartphone|
|2019-10-01 10:22:...|      view|   6000264|2053013560807654091|auto.accessories....|starline| 310.43|514453000|d3972d16-00c6-48d...|      alarm|
|2019-10-01 10:22:...|      view|  41900000|2090971686529663114|appliances.enviro...|    null|   23.6|551906402|a8f2500d-1af3-495...|     vacuum|
|2019-10-01 10:22:...|      view|  19200082|2053013556202308035|construction.tool...|  p.i.t.| 171.43|546270188|2f7b4975-521

In [0]:
df_with_productType.select('productType').groupBy('productType').count().orderBy('count',ascending=False).show(truncate=False)

+-------------+------+
|productType  |count |
+-------------+------+
|smartphone   |136625|
|clocks       |19173 |
|notebook     |14532 |
|headphone    |13257 |
|tv           |11457 |
|refrigerators|8806  |
|washer       |8357  |
|shoes        |8225  |
|vacuum       |8193  |
|player       |5477  |
|keds         |5281  |
|desktop      |5128  |
|air_heater   |4468  |
|subwoofer    |4015  |
|bed          |3901  |
|tablet       |3834  |
|drill        |3470  |
|cabinet      |3070  |
|telephone    |2971  |
|alarm        |2483  |
+-------------+------+
only showing top 20 rows



In [0]:
# CHecking in for the number of sessions 
#count the unique number and total number of sessions
#
df_with_productType.select('user_session').groupBy('user_session').count().orderBy('count', ascending=False).show(truncate=False)

+------------------------------------+-----+
|user_session                        |count|
+------------------------------------+-----+
|0e279033-f15a-4fc0-9bd8-b0cae4675e1d|163  |
|53d20c98-a4c7-42bb-906e-c6cd0bbd54c5|150  |
|a7081bbb-82b3-4917-9e20-da3af7e00d59|137  |
|75600132-2b71-45bd-b91e-2b7681b6cd89|131  |
|b42e946d-ae0c-49c0-a2ae-df842ab7e7c8|117  |
|e6a4a608-85b6-46d9-bf45-4594b36d8a74|114  |
|48ced088-be42-4436-a9ea-5be0a90be80b|109  |
|0c4a0195-84e9-49d6-8ccb-9a4ee43c2020|104  |
|19591d99-593a-4328-a02d-ce9a4f2cf0c8|103  |
|5f9abdf8-0651-4e92-a205-807737586704|100  |
|c8e69bcd-c368-4481-9bc0-517affe370f0|97   |
|04468175-8346-416e-939a-500505db8ef6|97   |
|bed19044-d233-4511-9fe0-d34c5bafe21a|96   |
|f9642ef1-235c-4513-abce-beb824d5e245|95   |
|238c95a3-f818-44ad-a5d6-bb59c8a2a847|92   |
|8bbe4515-d25e-4b28-b516-bc6c229b1b85|89   |
|9484f0f3-c272-4f4a-8e96-80f667982ff1|89   |
|9be95029-9bae-4666-845a-e9c9096f8408|89   |
|67f87a7e-201f-495a-a204-81b53633fa15|88   |
|bbf60bda-

In [0]:
# from this we know that each user has different user_session but that session changes when the user come back to app after a long time.
df_with_productType.select('user_id').where(col('user_session')=='0e279033-f15a-4fc0-9bd8-b0cae4675e1d').distinct().show()

+---------+
|  user_id|
+---------+
|516228618|
+---------+



In [0]:
df_with_productType.select('*').where(col('user_session')=='0e19508d-a698-409f-879f-ebd99bd762a6').show(truncate=False)

+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|event_time             |event_type|product_id|category_id        |category_code         |brand  |price |user_id  |user_session                        |productType|
+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|2019-10-01 10:22:44 UTC|purchase  |1004863   |2053013555631882655|electronics.smartphone|samsung|174.45|512599840|0e19508d-a698-409f-879f-ebd99bd762a6|smartphone |
+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+



In [0]:
# The proof that the new session for same user is created when user returns to the platform after an hour.
df_with_productType.select('*').where(col('user_id')=='512599840').show(truncate=False)

+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|event_time             |event_type|product_id|category_id        |category_code         |brand  |price |user_id  |user_session                        |productType|
+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|2019-10-01 10:22:44 UTC|purchase  |1004863   |2053013555631882655|electronics.smartphone|samsung|174.45|512599840|0e19508d-a698-409f-879f-ebd99bd762a6|smartphone |
|2019-10-01 11:27:33 UTC|view      |1004833   |2053013555631882655|electronics.smartphone|samsung|174.38|512599840|6a2e52b3-63de-4937-8fe3-5d417bb14d3f|smartphone |
+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+



In [0]:
# I am thinking of making a basket by grouping by the user id and on the same day, I cant do it on the basis of session id, 

#but lets see what if we do on same session id.


df_test = df_with_productType.select('*').where(col('event_type')=='purchase')
# df_test.show(truncate=False)

import pyspark.sql.functions as f
#df.groupby("col1").agg(f.concat_ws(", ", f.collect_list(df.col2)))
#df_test.select('productType').groupBy('user_session').agg(f.concat_ws(",", f.collect_list(df_test.productType))).show(truncate=False)

#df_test.select('*').show() 

#Working Code
df_test.select(col('user_session'), col('productType')).groupBy('user_session').agg(f.concat_ws(",",f.collect_list(df_test.productType))).show(truncate=False)


+------------------------------------+-------------------------------------------+
|user_session                        |concat_ws(,, collect_list(productType))    |
+------------------------------------+-------------------------------------------+
|000fdfe4-e1f0-4a93-9c22-f04066ad895e|clocks                                     |
|001226df-03c7-41e6-a9c4-7d0233348bd5|smartphone                                 |
|003c3709-b084-464f-9cd7-4baef596a8e6|oven,oven,oven                             |
|0042dd5a-8850-4752-a760-c24e5b0b043e|smartphone                                 |
|0043b973-66ee-4edb-b64c-6009c3a61928|smartphone                                 |
|00546b5e-b576-4028-84cc-dcefa8bd91ec|headphone                                  |
|0056e55b-b2fe-4b90-bb04-b71f6372cb74|smartphone                                 |
|006d417b-7eaf-4192-98ee-6afbcd7824da|smartphone                                 |
|00b3ff23-02b4-4064-94f8-35be78916981|smartphone                                 |
|00b

In [0]:
df_test.select('*').where(col('user_session')=='01216c46-646d-4e1f-b896-015c6101f8bf').show(truncate=False)

+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|event_time             |event_type|product_id|category_id        |category_code         |brand  |price |user_id  |user_session                        |productType|
+-----------------------+----------+----------+-------------------+----------------------+-------+------+---------+------------------------------------+-----------+
|2019-10-01 13:09:35 UTC|purchase  |1004750   |2053013555631882655|electronics.smartphone|samsung|203.35|555599569|01216c46-646d-4e1f-b896-015c6101f8bf|smartphone |
|2019-10-01 13:13:30 UTC|purchase  |1004750   |2053013555631882655|electronics.smartphone|samsung|203.35|555599569|01216c46-646d-4e1f-b896-015c6101f8bf|smartphone |
|2019-10-01 13:14:17 UTC|purchase  |1004750   |2053013555631882655|electronics.smartphone|samsung|203.35|555599569|01216c46-646d-4e1f-b896-015c6101f8bf|smartphone |
|2019-10-0

In [0]:
grouped_transactions1 = df_test.select('*').groupBy('user_session').agg(f.concat_ws(",",f.collect_list(df_test.productType))).select(col('user_session'), col("concat_ws(,, collect_list(productType))").alias("transactions"))
grouped_transactions1=grouped_transactions1.join(df_test, grouped_transactions.user_session==df_test.user_session, 'left')

grouped_transactions1.select(col('event_time'), col('user_id'), col('productType'), col('transactions')).show(truncate=False)

+-----------------------+---------+-----------+-------------------------------------------+
|event_time             |user_id  |productType|transactions                               |
+-----------------------+---------+-----------+-------------------------------------------+
|2019-10-01 13:37:06 UTC|520206104|clocks     |clocks                                     |
|2019-10-01 14:13:11 UTC|516675926|smartphone |smartphone                                 |
|2019-10-01 14:46:57 UTC|514242794|oven       |oven,oven,oven                             |
|2019-10-01 14:45:55 UTC|514242794|oven       |oven,oven,oven                             |
|2019-10-01 14:44:16 UTC|514242794|oven       |oven,oven,oven                             |
|2019-10-01 13:31:01 UTC|554093290|smartphone |smartphone                                 |
|2019-10-01 11:45:15 UTC|537131755|smartphone |smartphone                                 |
|2019-10-01 10:43:41 UTC|518088595|headphone  |headphone                        

In [0]:
grouped_transactions1.withColumnRenamed("concat_ws(,, collect_list(productType))", 'transactions')
grouped_transactions1.schema

Out[85]: StructType(List(StructField(user_session,StringType,true),StructField(concat_ws(,, collect_list(productType)),StringType,false),StructField(event_time,StringType,true),StructField(event_type,StringType,true),StructField(product_id,IntegerType,true),StructField(category_id,LongType,true),StructField(category_code,StringType,true),StructField(brand,StringType,true),StructField(price,FloatType,true),StructField(user_id,IntegerType,true),StructField(user_session,StringType,true),StructField(productType,StringType,true)))

In [0]:
# One proposal, the MBA is done on the things mostly purchased, so I think we should split the data according to the event type then we can make a transactional basket as per the people purchased the things. 

#THere are chances that event add_to_cart does not add value to MBA but be significant to understand other customer behavior

#View can be used in MBA.


#  data -> compress in new format -> read the compressed data ->  make_newdf_ignoreMissing -> separate_product_types -> split_as_per_events -> groupby_userID&date -> combine_into_transaction


In [0]:
! ls

conf  eventlogs  ganglia  logs	metastore_db  preload_class.lst


In [0]:
# NOTES
# lit method is used to impute a constant value to any column whereas when function is used to impute conditional values 
# udf = udf.withColumn('new_column', F.lit('constant_value'))
# udf = udf.withColumn('new_column', F.when(col('colum_name')=='some_value',0).when(col('column_name')=='some_value2',1).otherwise(2))


In [0]:
# Creating a UDF for transforming the timestamp to date and time
grouped_transactions.select('event_time').show(5, truncate=False)

+-----------------------+
|event_time             |
+-----------------------+
|2019-10-01 10:22:44 UTC|
|2019-10-01 10:22:44 UTC|
|2019-10-01 10:22:55 UTC|
|2019-10-01 10:22:50 UTC|
|2019-10-01 10:25:01 UTC|
+-----------------------+
only showing top 5 rows



In [0]:
#ERROR
# def convert_time(time):
#     time = time.split(' ')[:2]
#     time = time[0]+' '+time[1]
#     timenew = F.to_timestamp(time, 'yyyy-MM-dd HH:mm:ss')
#     return timenew

# convert_time('2019-10-01 10:22:44 UTC')
# from pyspark.sql.functions import to_timestamp
# time_udf = F.udf(convert_time, TimestampType())
# nonNAdf = nonNAdf.withColumn('eventTime', time_udf('event_time'))
# nonNAdf.show()

In [0]:
def convert_time(time):
    return(F.to_timestamp(time))
convert_time('2019-10-01 10:22:44 UTC')

time_udf = F.udf(convert_time, TimestampType())
nonNAdf = nonNAdf.withColumn('eventTime', time_udf('event_time'))

In [0]:
nonNAdf.show()

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
<command-34604069342590> in <module>
----> 1 nonNAdf.show()

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    500 
    501         if isinstance(truncate, bool) and truncate:
--> 502             print(self._jdf.showString(n, 20, vertical))
    503         else:
    504             try:

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came from that shows a non-Pythonic
    122                 # JVM exception message.
--> 123   

In [0]:
from pyspark.sql.functions import to_timestamp
df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
df.select(to_timestamp(df.t, 'yyyy-MM-dd HH:mm:ss').alias('dt')).collect()

Out[9]: [Row(dt=datetime.datetime(1997, 2, 28, 10, 30))]

In [0]:
# Saving file to any format here it is parquet
parquet_path= ''
df.write.format("parquet").save(parquet_path)